In [1]:
from flask import Flask, redirect, request, render_template, url_for, send_from_directory, make_response
from werkzeug.wrappers import Request, Response
from werkzeug.serving import run_simple
from jsonrpc import JSONRPCResponseManager, dispatcher
import os
import hashlib
import pandas as pd
from werkzeug.utils import secure_filename
import pickle

In [ ]:
UPLOAD_FOLDER = '\\Datasets'
ALLOWED_EXTENSIONS = set(['csv','txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'])

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = os.path.join(app.instance_path)

In [ ]:
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS 
    
@app.route('/', methods=['GET','POST'])
def upload_file():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        global file_name = '/tmp/' + file.filename
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            df = pd.read_csv(file)
            resp = make_response(df.to_csv())
            resp.headers["Content-Disposition"] = "attachment; filename=export.csv"
            resp.headers["Content-Type"] = "text/csv"
            return resp  

    return redirect(url_for('predict'))

#getting our trained model
model = pickle.load(open("tag_model", "rb"))

@app.route('/predict', methods=['GET', 'POST'])
def predict():
    #getting the untagged dataset from the request body
    original_dataset = request.get_json()['input']
    #use our model to predict tags to each column
    prediction = model.predict(original_dataset).tolist()
    #setting 
    response = {}
    response['predictions'] = prediction
    
    return flask.jsonify(response)
    


In [ ]:

#create the route for file uploading
# @app.route('/predict', methods=['GET', 'POST'])
# def upload_file():
#     if request.method == 'POST':
#         if 'file' not in request.files:
#             flash('No file part')
#             return redirect(request.url)
#         file = request.files['file']
#         if file:
#             original_dataset = file.read()
#         #preparing a response object and storing our model's predictions
#         response = {}
#         response['predictions'] = model.predict(original_dataset).tolist()

#     #returning the response object as json
#     return flask.jsonify(response)    
